In [86]:
import json
import json
import random
import random

documents = []
# Open the JSONL file for reading
with open('train.jsonl', 'r') as jsonl_file:
    # Iterate over the lines in the file
    for line in jsonl_file:
        # Parse each line as a JSON object
        data = json.loads(line)
        text = data['text']
        starting_point = 0
        entities = []
        for entry in data['entities']:
            substring = entry['surfaceform']
            
            start_index = text.find(substring)
            # if (start_index < starting_point) & (start_index != -1):
            #     temp_start = text[starting_point:].find(substring)
            #     print(temp_start)
            #     print(start_index)
            #     end_index = temp_start + len(substring)
            #     print(text[temp_start+starting_point:end_index])
            
            if start_index == -1:
                continue
            end_index = start_index + len(substring)
            starting_point = end_index
            
            if start_index != entry['boundaries'][0]:
                entry['boundaries'] = [start_index, end_index]
            
            entities.append(entry)

        data['entities'] = entities
with open("output_new.jsonl", "a") as f:
    json.dump(data, f)
    f.write('\n')

60
72

-1
85



In [24]:
def count_relations(file_name:str):
    relations_my = []
    # Open the JSONL file for reading
    with open('train.jsonl', 'r') as jsonl_file:
        # Iterate over the lines in the file
        for line in jsonl_file:
            # Parse each line as a JSON object
            # print(line)
            data = json.loads(line)

            for entry in data['triples']:
                relations_my.append(entry['predicate']['surfaceform'])

    return relations_my

In [25]:
from collections import Counter
import extract_data as ex

relations_counter = Counter(ex.count_relations('train.jsonl'))

In [27]:
import pandas as pd
relations_counter_df = pd.DataFrame.from_dict(relations_counter, orient='index')

In [33]:
relations_df = pd.read_csv('./relations_count.tsv', header = None, sep='\t')
relations = list(relations_df[0])

In [37]:
relations_counter_df = relations_counter_df.reset_index()

In [40]:
relations_counter_df.columns = ['word', 'count']

In [41]:
relations_df.columns = ['word', 'count']

In [43]:
relations_df_new = pd.concat([relations_counter_df, relations_df], axis=0)

In [46]:
relations_df_new.to_csv('relations_count.tsv', sep='\t', header=False, index=False)

In [13]:
import json
import random
import random

documents = []
relations_my = []
id_entity = {}
entities = []
triplets = []
# Open the JSONL file for reading
with open('chev_data2.json', 'r') as jsonl_file:
    # Iterate over the lines in the file
    data = json.load(jsonl_file)
    # print(data['annotations'])
    for item in data:
        document = {}
        print(item['id'])
        doc_id = random.randint(1, 1000000)
        document['doc_id'] = doc_id
        document['uri'] = item['id']
        text = item['data']['text']
        document['text'] = text
        title = ' '.join(text.split(' ')[:5])
        document['title'] = title

        # Parse each line as a JSON object
        for entry in item['annotations']:
            for res in entry['result']:
                try:
                    entity = {}
                    boundaries = []
                    
                   
                    if res['type'] == 'labels':
                        uri = res['id']
                        value = res['value']
                        surfaceform = value['text']
                        boundaries.append(value['start'])
                        boundaries.append(value['end'])
                        entity['uri'] = uri
                        entity['surfaceform'] = surfaceform
                        entity['boundaries'] = boundaries
                        entity['annotator'] = 'Me'

                        id_entity[uri] = entity
                        entities.append(entity)

                    elif res['type'] == 'relation':
                        triplet = {}
                        from_id = res['from_id']
                        to_id = res['to_id']
                        surfaceform = res['labels'][0]
                        predicate = {}
                        uri_pred_len = min(len(from_id), len(to_id))
                        sample_list = random.sample(from_id+to_id, k=uri_pred_len)
                        # Join the list of characters into a single string
                        sample_string = ''.join(sample_list)

                        predicate['uri'] = sample_string
                        predicate['annotator'] = 'Me'
                        predicate['surfaceform'] = surfaceform
                        predicate['boundaries'] = None

                        relations_my.append(surfaceform)

                        if res['direction'] == 'right':
                            triplet['subject'] = id_entity[from_id]
                            triplet['object'] = id_entity[to_id]
                            triplet['predicate'] = predicate

                            triplets.append(triplet)
                        else:
                            print(res)
                except Exception as e:
                    print(res)
                    print(e)
                    break
        
        document['entities'] = entities
        document['triples'] = triplets
        documents.append(document)


1
14
15


In [14]:
from collections import Counter

Counter(relations_my)

Counter({'protect': 2,
         'support': 8,
         'reduce': 7,
         'request': 2,
         'aim': 5,
         'stop': 3,
         'of': 3,
         'protect against': 3,
         'is': 1,
         'from': 1,
         'is required': 4,
         'identify': 2})

In [15]:
len(entities)

141

In [6]:
def get_entites_spacy(file_name):
    documents = []
    # Open the JSONL file for reading
    with open(file_name, 'r') as jsonl_file:
        # Iterate over the lines in the file
        data = json.load(jsonl_file)
        for item in data:
            text = item['data']['text']

            entities = []
            # Parse each line as a JSON object
            for entry in item['annotations']:
                for res in entry['result']:
                    try:
                        boundaries = []
                    
                        if res['type'] == 'labels':
                            value = res['value']
                            from_name = res['from_name']
                            ner = from_name.split('-')[1]
                            if ner == 'General':
                                continue
                            boundaries.append(value['start'])
                            boundaries.append(value['end'])
                            entities.append((boundaries[0], boundaries[1], ner))
                    except Exception as e:
                        print(res)
                        print(e)
                        break
            
            documents.append((text, {'entities': entities}))
    return documents

In [88]:
from extract_data import get_entites_spacy
documents = []
documents += get_entites_spacy('chev_data.json')
documents += get_entites_spacy('chev_data2.json')
print(len(documents))

6


In [1]:
import extract_data as ex
documents = []
documents += ex.get_relations_from_label_studio_data('./chev_data.json')
documents += ex.get_relations_from_label_studio_data('./chev_data2.json')
len(documents)

223
{'from_id': 'gm2nXX5EIp', 'to_id': '_vpFR1OHM7', 'type': 'relation', 'direction': 'right', 'labels': []}
list index out of range
230
{'from_id': 'fSjdyI5ORI', 'to_id': 'gzXgGzbIg7', 'type': 'relation', 'direction': 'right', 'labels': []}
list index out of range
231
1
14
15


6

In [3]:
import json
# open file for writing
with open("output.jsonl", "a") as f:
    # write each object as a line in the file
    for obj in documents:
        json.dump(obj, f)
        f.write('\n')

In [ ]:
# TRAIN_DATA = [
#     ("Apple is looking at buying U.K. startup for $1 billion", {"entities": [(0, 5, "ORG"), (27, 31, "GPE"), (44, 52, "MONEY")]}),
#     ("Google acquires Kaggle for undisclosed amount", {"entities": [(0, 6, "ORG"), (17, 23, "PRODUCT"), (31, 38, "MONEY")]}),
#     # Add more examples of your new entity classes here
# ]

sentences = ["Apple is looking at buying U.K. startup for $1 billion", "Google acquires Kaggle for undisclosed amount"] # Add more examples of your new entity classes here
labels = [[0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0]] # Convert entity labels to integers


In [18]:
relations_df = pd.read_csv('./relations_count.tsv', header = None, sep='\t')
relations = list(relations_df[0])
print(relations)

['country', 'located in the administrative territorial entity', 'contains administrative territorial entity', 'date of birth', 'sport', 'instance of', 'point in time', 'date of death', 'part of', 'shares border with', 'publication date', 'place of birth', 'inception', 'member of sports team', 'genre', 'performer', 'has part', 'cast member', 'country of citizenship', 'participant in', 'parent taxon', 'subclass of', 'location', 'member of political party', 'director', 'author', 'educated at', 'capital', 'league', 'headquarters location', 'record label', 'position held', 'country of origin', 'spouse', 'member of', 'conflict', 'followed by', 'sibling', 'located in or next to body of water', 'owned by', 'father', 'notable work', 'occupation', 'follows', 'publisher', 'part of the series', 'mouth of the watercourse', 'award received', 'child', 'mountain range', 'operator', 'position played on team / speciality', 'winner', 'sports season of league or competition', 'place of death', 'manufactur

In [49]:
import json
import regex as re
import pandas as pd

relations_df = pd.read_csv('./relations_count.tsv', header = None, sep='\t')
relations = list(relations_df[0])
print(relations)
with open('./train.jsonl', encoding="utf-8") as f:
    for id_, row in enumerate(f):
        if id_ >= 1000:
            print(id)
            break
        article = json.loads(row)
        prev_len = 0
        # print(article)
        if len(article['triples']) == 0:
            continue
        count = 0
        for text_paragraph in article['text'].split('\n'):
            if len(text_paragraph) == 0:
                continue
            # print(text_paragraph)
            sentences = re.split(r'(?<=[.])\s', text_paragraph)
            text = ''
            for sentence in sentences:
                text += sentence + ' '
                if any([entity['boundaries'][0] < len(text) + prev_len < entity['boundaries'][1] for entity in article['entities']]):
                    continue
                entities = sorted([entity for entity in article['entities'] if prev_len < entity['boundaries'][1] <= len(text)+prev_len], key=lambda tup: tup['boundaries'][0])
                decoder_output = '<triplet> '
                # print(entities)
                for int_ent, entity in enumerate(entities):
                    # for triplet in article['triples']:
                    #     print(triplet)
                    #     print(prev_len < triplet['subject']['boundaries'][1] <=len(text) + prev_len)
                    #     print(prev_len < triplet['object']['boundaries'][1] <=len(text) + prev_len)
                    #     print(triplet['predicate']['surfaceform'] in relations)
                    triplets = sorted([triplet for triplet in article['triples'] 
                                       if triplet['subject'] == entity and prev_len< triplet['subject']['boundaries'][1]<=len(text) + prev_len and prev_len< triplet['object']['boundaries'][1]<=len(text) + prev_len and triplet['predicate']['surfaceform'] in relations], key=lambda tup: tup['object']['boundaries'][0])
                    # print(triplets)
                    if len(triplets) == 0:
                        continue
                    decoder_output += entity['surfaceform'] + ' <subj> '
                    for triplet in triplets:
                        decoder_output += triplet['object']['surfaceform'] + ' <obj> '  + triplet['predicate']['surfaceform'] + ' <subj> '
                    decoder_output = decoder_output[:-len(' <subj> ')]
                    decoder_output += ' <triplet> '
                decoder_output = decoder_output[:-len(' <triplet> ')]
                count += 1
                prev_len += len(text)
                print(decoder_output)
                if len(decoder_output) == 0:
                    text = ''
                    continue

                text = re.sub('([\[\].,!?()])', r' \1 ', text.replace('()', ''))
                text = re.sub('\s{2,}', ' ', text)


                text = ''


['aim', 'is', 'reduce', 'request', 'report', 'concern', 'fail', 'from', 'adopt', 'is responsible', 'is required', 'country', 'located in the administrative territorial entity', 'contains administrative territorial entity', 'date of birth', 'sport', 'instance of', 'point in time', 'date of death', 'part of', 'shares border with', 'publication date', 'place of birth', 'inception', 'member of sports team', 'genre', 'performer', 'has part', 'cast member', 'country of citizenship', 'participant in', 'parent taxon', 'subclass of', 'location', 'member of political party', 'director', 'author', 'educated at', 'capital', 'league', 'headquarters location', 'record label', 'position held', 'country of origin', 'spouse', 'member of', 'conflict', 'followed by', 'sibling', 'located in or next to body of water', 'owned by', 'father', 'notable work', 'occupation', 'follows', 'publisher', 'part of the series', 'mouth of the watercourse', 'award received', 'child', 'mountain range', 'operator', 'positio